1) Creating/Opening/Closing a netCDF file.

In [2]:
from netCDF4 import Dataset

rootgrp = Dataset("test.nc", "w", format="NETCDF4")
print (rootgrp.data_model)

NETCDF4


In [3]:
rootgrp.close()

2) Groups in a netCDF file.

In [4]:
rootgrp = Dataset("test.nc", "a")
fcstgrp = rootgrp.createGroup("forecasts")
analgrp = rootgrp.createGroup("analyses")

In [5]:
print (rootgrp.groups)

OrderedDict([('forecasts', <class 'netCDF4._netCDF4.Group'>
group /forecasts:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
), ('analyses', <class 'netCDF4._netCDF4.Group'>
group /analyses:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
)])


3) Dimensions in a netCDF file.

In [6]:
level = rootgrp.createDimension("level", None)
time = rootgrp.createDimension("time", None)
lat = rootgrp.createDimension("lat", 73)
lon = rootgrp.createDimension("lon", 144)

In [7]:
print (rootgrp.dimensions)

OrderedDict([('level', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'level', size = 0
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0
), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73
), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 144
)])


4) Variables in a netCDF file.

In [8]:
times = rootgrp.createVariable("time","f8",("time",))
levels = rootgrp.createVariable("level","i4",("level",))
latitudes = rootgrp.createVariable("lat","f4",("lat",))
longitudes = rootgrp.createVariable("lon","f4",("lon",))

temp = rootgrp.createVariable("temp","f4",("time","level","lat","lon",))

In [11]:
print (latitudes)

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (73,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [12]:
print (levels)

<class 'netCDF4._netCDF4.Variable'>
int32 level(level)
unlimited dimensions: level
current shape = (0,)
filling on, default _FillValue of -2147483647 used



In [13]:
print(temp)

<class 'netCDF4._netCDF4.Variable'>
float32 temp(time, level, lat, lon)
unlimited dimensions: time, level
current shape = (0, 0, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used



In [14]:
print (rootgrp.variables)

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
unlimited dimensions: time
current shape = (0,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('level', <class 'netCDF4._netCDF4.Variable'>
int32 level(level)
unlimited dimensions: level
current shape = (0,)
filling on, default _FillValue of -2147483647 used
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (73,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('lon', <class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (144,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('temp', <class 'netCDF4._netCDF4.Variable'>
float32 temp(time, level, lat, lon)
unlimited dimensions: time, level
current shape = (0, 0, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used
)])


5) Attributes in a netCDF file.

In [15]:
import time

rootgrp.description = "bogus example script"
rootgrp.history = "Created " + time.ctime(time.time())
rootgrp.source = "netCDF4 python module tutorial"

latitudes.units = "degrees north"
longitudes.units = "degrees east"
levels.units = "hPa"
temp.units = "K"
times.units = "hours since 0001-01-01 00:00:00.0"
times.calendar = "gregorian"

In [16]:
for name in rootgrp.ncattrs():
    print ("Global attr", name, "=", getattr(rootgrp,name))

Global attr description = bogus example script
Global attr history = Created Tue Aug 13 09:58:16 2019
Global attr source = netCDF4 python module tutorial


6) Writing data to and retrieving data from a netCDF variable.

In [17]:
import numpy

lats =  numpy.arange(-90,91,2.5)
lons =  numpy.arange(-180,180,2.5)

latitudes[:] = lats
longitudes[:] = lons
print ("latitudes =\n",latitudes[:])

latitudes =
 [-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90. ]


In [18]:
# append along two unlimited dimensions by assigning to slice.
nlats = len(rootgrp.dimensions["lat"])
nlons = len(rootgrp.dimensions["lon"])
print ("temp shape before adding data = ",temp.shape)

from numpy.random import uniform
temp[0:5,0:10,:,:] = uniform(size=(5,10,nlats,nlons)) # time, level, lat, lon
print ("temp shape after adding data = ",temp.shape)

# levels have grown, but no values yet assigned.
print ("levels shape after adding pressure data = ",levels.shape)

temp shape before adding data =  (0, 0, 73, 144)
temp shape after adding data =  (5, 10, 73, 144)
levels shape after adding pressure data =  (10,)


In [19]:
# now, assign data to levels dimension variable.
levels[:] =  [1000.,850.,700.,500.,300.,250.,200.,150.,100.,50.]

7) Dealing with time coordinates.

In [21]:
# fill in times.
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num

dates = [datetime(2001,3,1)+n*timedelta(hours=12) for n in range(temp.shape[0])]
times[:] = date2num(dates,units=times.units,calendar=times.calendar)
print ("time values (in units %s): " % times.units+"\n",times[:])

dates = num2date(times[:],units=times.units,calendar=times.calendar)
print ("dates corresponding to time values:\n",dates)

time values (in units hours since 0001-01-01 00:00:00.0): 
 [17533104. 17533116. 17533128. 17533140. 17533152.]
dates corresponding to time values:
 [real_datetime(2001, 3, 1, 0, 0) real_datetime(2001, 3, 1, 12, 0)
 real_datetime(2001, 3, 2, 0, 0) real_datetime(2001, 3, 2, 12, 0)
 real_datetime(2001, 3, 3, 0, 0)]
